In [1]:
import os
import sys
from pathlib import Path
base_dir = Path(os.getcwd()).resolve().parent
sys.path.append(str(base_dir))

# General testing

Basic tensor operations and ensure that the .backward functionality works

In [2]:
# Import things
from src import Tensor

In [3]:
# Testing some tensor operations
a = Tensor([1,2,3])
b = Tensor([1,2,3])
c = a+b
c

tensor([2 4 6], requires_grad=False)

In [4]:
d = a*b
d

tensor([1 4 9], requires_grad=False)

In [5]:
c.backward()
print(f"a.grad: {a.grad}")
print(f"b.grad: {b.grad}")
print(f"c.grad: {c.grad}")

print("---"*5)

d.backward()
print(f"a.grad: {a.grad}")
print(f"b.grad: {b.grad}")
print(f"d.grad: {d.grad}")


a.grad: [0. 0. 0.]
b.grad: [0. 0. 0.]
c.grad: [1 1 1]
---------------
a.grad: [0. 0. 0.]
b.grad: [0. 0. 0.]
d.grad: [1 1 1]


Everything is working fine for now, lets test the NN functionality and create a NN.

In [6]:
# Testing the matmul stuff too
a = Tensor([[1,2,3],
            [4,5,6]])
b = Tensor([[1,2,3],
            [4,5,6]])
c = a@b.T
c.backward()
print(a.grad)
print(b.grad)

[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]


# Training on MNIST

## Testing training stuff

In [7]:
# Load dataset and prep it (kinda)
import numpy as np
from typing import Literal, Generator

# Load from dir
train_data = np.loadtxt("./data/train.csv", delimiter=",", skiprows=1)  # skip header
test_data = np.loadtxt("./data/test.csv", delimiter=",", skiprows=1)  # skip header
print(f"train: {train_data.shape} | test: {test_data.shape}")

# Normalize
X_train = train_data[:, 1:] / 255.
y_train = train_data[:, 0].astype(int)
X_test = test_data[:, 1:] / 255.
y_test = test_data[:, 0].astype(int)
print(f"normalized data-splits")

class Dataset:
    def __init__(self, split: Literal["train", "test"], batch_size: int = 1):
        self.X_split = X_train if split == "train" else X_test
        self.y_split = y_train if split == "train" else y_test
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.y_split) / self.batch_size))

    def __getitem__(self, idx: int):
        start = idx * self.batch_size
        end = min((idx + 1) * self.batch_size, len(self.y_split))

        batch_X = self.X_split[start:end]
        batch_y = self.y_split[start:end]

        return Tensor(batch_X), Tensor(batch_y)  # Targets can stay as np.ndarray for now
    
train_dataset = Dataset(split="train")
test_dataset = Dataset(split="test")
print(f"train_dataset: {len(train_dataset)} | test_dataset: {len(test_dataset)}")

X, y = train_dataset[np.random.randint(low=0, high=len(train_dataset))]
print(f"Get random samples | X: {X.shape} | y: {y}")

train: (60000, 785) | test: (10000, 785)
normalized data-splits
train_dataset: 60000 | test_dataset: 10000
Get random samples | X: (1, 784) | y: tensor([5], requires_grad=False)


In [8]:
# Creating a sample NN
from src import nn
model = nn.Sequential(
    nn.Linear(in_features=784, out_features=100),
    nn.Linear(in_features=100, out_features=100),
    nn.Linear(in_features=100, out_features=20),
    nn.Linear(in_features=20, out_features=10),
)

print(model.layers)
print("---"*5)

from src.nn import functional as F
logits = model(X)
print(f"logits: {logits}")
print("---"*5)

loss = F.cross_entropy_loss(logits, y)
print(f"loss: {loss}")
print("---"*5)

# Check if loss value is right
print(-np.log(1/10))
print("---"*5)

[<src.nn.linear.Linear object at 0x10675a420>, <src.nn.linear.Linear object at 0x10468f620>, <src.nn.linear.Linear object at 0x10468ff80>, <src.nn.linear.Linear object at 0x1067598b0>]
---------------
logits: tensor([[-0.01122063 -0.07404669 -0.46543919 -0.23543532  0.26150321  0.0727318
  -0.33352489  0.2659135   0.02230216 -0.16454536]], requires_grad=True)
---------------
loss: tensor(2.189211274477429, requires_grad=True)
---------------
2.3025850929940455
---------------


Very close, good. There is some entropy from random weight init, but that fine and looks like we are ready to train.

In [9]:
# Before that, lets prform a loss.backward and see if thats working too
loss.backward()

In [10]:
import graphviz
from graphviz import Digraph
from collections import defaultdict

def trace(root):
    """
    Traverses the compute graph from the root tensor and collects all nodes and edges.
    """
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root):
    """
    Generates a graphviz Digraph object for a given root tensor.
    Each node represents a tensor, and edges represent the flow of computation.
    """
    dot = Digraph(comment='Compute Graph', graph_attr={'rankdir': 'LR'})  # LR = Left to Right

    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        
        # Create a node for the tensor
        dot.node(
            name=uid, 
            label=f"{{ shape: {n.shape} | operand: {n._operand} | grad: {n.grad.round(2) if n.ndim == 0 else n.grad.round(2).flatten()} }}",
            shape='record',
            style='filled',
            fillcolor='lightblue' if n.requires_grad else 'lightgray'
        )

        # Create a node for the operation if it exists
        if n._operand:
            dot.node(name=uid + n._operand, label=n._operand, shape='circle', style='filled', fillcolor='lightyellow')
            dot.edge(uid + n._operand, uid)
        
    for n1, n2 in edges:
        # Connect children to the operation that produced n2
        dot.edge(str(id(n1)), str(id(n2)) + n2._operand)

    return dot

# # Visualize the graph
dot_graph = draw_dot(loss)
dot_graph.render('compute_graph_example', view=True) # This will create a file and open it

# # You can also directly view the graph in environments that support it
display(dot_graph)


In [11]:
for param in model.parameters():
    print(param.grad)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[ 0.01446386  0.00480641  0.03006767 ...  0.00708178  0.00979378
   0.03558179]
 [-0.00171026 -0.00056833 -0.00355531 ... -0.00083738 -0.00115805
  -0.00420732]
 [-0.00266887 -0.00088688 -0.00554807 ... -0.00130673 -0.00180714
  -0.00656554]
 ...
 [ 0.00134629  0.00044738  0.00279869 ...  0.00065917  0.0009116
   0.00331194]
 [-0.00445061 -0.00147896 -0.00925199 ... -0.0021791  -0.0030136
  -0.01094871]
 [-0.01316141 -0.0043736  -0.02736012 ... -0.00644407 -0.00891186
  -0.0323777 ]]
[[-0.0007242  -0.00241862  0.00151947 ...  0.00123624 -0.00056958
  -0.00180882]
 [-0.01920756 -0.06414745  0.04029994 ...  0.0327881  -0.01510671
  -0.04797422]
 [ 0.00191001  0.00637886 -0.00400745 ... -0.00326047  0.00150222
   0.00477058]
 ...
 [-0.01157688 -0.03866327  0.02428978 ...  0.01976221 -0.00910519
  -0.02891526]
 [ 0.00428827  0.01432151

Nice, no error

## Train run

In [12]:
model = nn.Sequential(
    nn.Linear(784, 128, bias=True),
    nn.ReLU(),
    nn.Linear(128, 10, bias=True),
)

#for param in model.parameters():
#    print(param.requires_grad, param.grad)

X.data[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [ ]:
# Training
from src import optim

def accuracy_fn(logits, targets):
    if isinstance(targets, Tensor):
        targets = targets.data
    preds = np.argmax(logits.data, axis=-1)
    return (preds == targets).astype(np.float32).mean()

epochs = 5
batch_size = 32
train_dataset = Dataset(split="train", batch_size=batch_size)

optimizer = optim.SGD(params=model.parameters(), lr=1e-2)
losses = []
accs = []

for epoch in range(epochs):
    running_loss = 0.0
    running_acc = 0.0
    indices = np.random.permutation(len(train_dataset))

    for i in indices:
        X, y = train_dataset[i]
        logits = model(X)
        loss = F.cross_entropy_loss(logits, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.data
        running_acc += accuracy_fn(logits, y)

    running_loss /= len(train_dataset)
    running_acc /= len(train_dataset)
    losses.append(running_loss)
    accs.append(running_acc)
    print(f"Epoch {epoch+1}/{epochs} | Loss: {running_loss:.4f} | Accuracy: {running_acc*100:.2f}")


Epoch 1/5 | Loss: 0.3517 | Accuracy: 90.16
Epoch 2/5 | Loss: 0.3034 | Accuracy: 91.48
Epoch 3/5 | Loss: 0.2738 | Accuracy: 92.31
Epoch 4/5 | Loss: 0.2512 | Accuracy: 93.00
Epoch 5/5 | Loss: 0.2319 | Accuracy: 93.55
